In [1]:
from pydrake.all import (
    StartMeshcat
    )
from time import sleep
from generate_models import Playground
from pydrake.systems.analysis import Simulator
from manipulation.utils import RenderDiagram
from robots import IIWA
import numpy as np
from planner import ActionExecutor, connect_to_the_world
from planner import *
from environment import MovableBody
from utils import choose_closest_heuristic

meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


In [4]:
playground = Playground(meshcat=meshcat, time_step=0.001)
iiwa = IIWA(playground.construct_welded_sim(playground.default_continuous_state()))

def get_final_plan(playground: Playground, mb: MovableBody):
    plant_context = iiwa.get_fresh_plant_context()    
    plan = MoveToGrasp(playground, mb).then(
        OpenGripper(playground)
        ).then(
        Grasp(playground, mb)
        ).then(
        CloseGripper(playground)
        ).then(
        PostGrasp(playground, mb)
        ).then(
        MoveToDeliver(playground, mb) 
        )
    # plan = PostGrasp(playground, mb)
    return plan


def simulate_env():
    mb = playground.env.movable_bodies[0]

    action_executor = ActionExecutor(get_final_plan(playground, mb))
    # action_executor.action.path
    sim_diagram = connect_to_the_world(playground, action_executor)

    simulator = Simulator(sim_diagram)
    # RenderDiagram(sim_diagram, max_depth=2)
    meshcat.StartRecording(set_visualizations_while_recording=True)
    simulator.AdvanceTo(5.0)
    meshcat.StopRecording()
    meshcat.PublishRecording()

    # while True:
    #     sleep(1)

    # return action_executor.action.path

path = simulate_env()

/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
IIWA self-collision filtering applied.
Plant init:  [ 0.5    0.6    0.    -1.75   0.     1.     0.    -0.025  0.025  1.
  0.     0.     0.     0.     0.     0.   ]
/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
IIWA self-collision filtering applied.
Iiwa initialized
composeed
composeed
composeed
composeed
composeed
  State output port plant size:  31
  State output port plant:  [ 0.5    0.6    0.    -1.75   0.     1.     0.    -0.025  0.025  1.
  0.     0.     0.     0.88   0.     0.64   0.     0.     0.     0.
  0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  0.   ]
/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
IIWA self-collision filtering applied.
Iiwa initialized
/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
IIWA self-collision filtering applied.
Action executor
CONNECT TO THE WORLD: 
iiwa_position_command <pydrake.systems.framework.LeafOutputPort object 

TypeError: 'NoneType' object is not subscriptable

In [7]:
import numpy as np
from pydrake.all import Simulator, DiagramBuilder, MeshcatVisualizer
from robots import IIWA
from generate_models import Playground
from planner.ActionExecutor import ActionExecutor
from planner.Command import Command

mb = playground.env.movable_bodies[0]

def simulate_path(playground, path):
    """
    Simulates the robot moving along the given path.
    """
    if not path or len(path) < 2:
        print("Path is empty or too short to simulate.")
        return
    
    iiwa = IIWA(playground.construct_welded_sim(playground.default_continuous_state()))
    command = Command(iiwa=iiwa, position_command=path[0])
    
    action_executor = ActionExecutor(MoveToGrasp(playground, mb))
    sim_diagram = connect_to_the_world(playground, action_executor)
    
    simulator = Simulator(sim_diagram)
    meshcat = playground.env.meshcat
    
    visualizer = MeshcatVisualizer.AddToBuilder(
        DiagramBuilder(), sim_diagram.GetOutputPort("query_object"), meshcat
    )
    
    # Start recording visualization
    meshcat.StartRecording(set_visualizations_while_recording=True)
    
    for q in path:
        command.position_command = q
        action_executor.command = command
        simulator.AdvanceTo(simulator.get_context().get_time() + 0.5)
    
    meshcat.StopRecording()
    meshcat.PublishRecording()
    print("Simulation complete.")

simulate_path(playground, path)

Path is empty or too short to simulate.


In [ ]:

def test_meshcat_visualization():
    

    # Start Meshcat
    # meshcat = StartMeshcat()

    # Initialize the Playground environment
    playground = Playground(meshcat=meshcat)

    
    iiwa = IIWA(playground.construct_welded_sim(playground.default_continuous_state()))

    
    # RenderDiagram(playground.diagram, max_depth=2)

    # Run a basic visualization test
    # print("Testing Meshcat visualization...")

    # Ensure the meshcat server is running
    assert meshcat is not None, "Meshcat did not start properly."

    # # Display the scene (Meshcat should render it)
    # playground.env.scene_graph.Publish(playground.env.diagram.CreateDefaultContext())

    # # Keep the visualization open for a short time
    simulator = Simulator(playground.env.diagram)
    simulator.set_target_realtime_rate(1.0)
    
    sim_context = simulator.get_mutable_context()

    # Get a mutable reference to the input port 
    iiwa_position_port = playground.diagram.GetInputPort("iiwa.position")
    wsg_position_port = playground.diagram.GetInputPort("wsg.position")

    # Set a custom input value (e.g., all joints at 0.5 radians)
    iiwa_position_port.FixValue(sim_context, np.full(7, 0.5))
    wsg_position_port.FixValue(sim_context, np.full(1, 0.06))
    

    simulator.AdvanceTo(0.01)

    print("Meshcat visualization should now be visible.")
    time.sleep(3)

    print("Meshcat visualization test completed.")

# Call the function to test Meshcat
test_meshcat_visualization()

/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
Plant init:  [ 0.5   0.6   0.   -1.75  0.    1.    0.    0.    0.    1.    0.    0.
  0.    0.    0.    0.  ]
Directives:  [ModelDirective(add_model=AddModel(file='package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf', name='iiwa', default_joint_positions={'iiwa_joint_1': array([0.]), 'iiwa_joint_2': array([0.6]), 'iiwa_joint_3': array([0.]), 'iiwa_joint_4': array([-1.75]), 'iiwa_joint_5': array([0.]), 'iiwa_joint_6': array([1.]), 'iiwa_joint_7': array([0.])}, default_free_body_pose={}), add_model_instance=None, add_frame=None, add_weld=None, add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=None, add_model_instance=None, add_frame=None, add_weld=AddWeld(parent='world', child='iiwa::iiwa_link_0', X_PC=None), add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=AddModel(file='package://manipulation/hydro/schunk_wsg_50_with_tip.sdf', name='wsg', defau

In [ ]:
print(action_executor.action.path)

In [3]:
def test_iiwa_functions(iiwa):
    print("Testing IIWA functions...\n")

    # 1. Test get_fresh_plant_context
    try:
        context = iiwa.get_fresh_plant_context()
        print("✅ get_fresh_plant_context() works")
    except Exception as e:
        print(f"❌ get_fresh_plant_context() failed: {e}")

    # 2. Test get_fresh_scene_graph_context
    try:
        context = iiwa.get_fresh_scene_graph_context()
        print("✅ get_fresh_scene_graph_context() works")
    except Exception as e:
        print(f"❌ get_fresh_scene_graph_context() failed: {e}")

    # 3nts(. Test get_robot_nominal_position
    try:
        nominal_q = iiwa.get_robot_nominal_position()
        print(f"✅ get_robot_nominal_position() works: {nominal_q}")
    except Exception as e:
        print(f"❌ get_robot_nominal_position() failed: {e}")

    # 4. Test num_actuated_joints
    try:
        num_actuated = iiwa.num_actuated_joints()
        print(f"✅ num_actuated_joints() works: {num_actuated}")
    except Exception as e:
        print(f"❌ num_actuated_joints() failed: {e}")

    # 5. Test num_joints
    try:
        num_joints = iiwa.num_joints()
        print(f"✅ num_joints() works: {num_joints}")
    except Exception as e:
        print(f"❌ num_joints() failed: {e}")

    # 6. Test get_bounded_position_lower_limit
    try:
        lower_limit = iiwa.get_bounded_position_lower_limit()
        print(f"✅ get_bounded_position_lower_limit() works: {lower_limit}")
    except Exception as e:
        print(f"❌ get_bounded_position_lower_limit() failed: {e}")

    # 7. Test get_bounded_position_upper_limit
    try:
        upper_limit = iiwa.get_bounded_position_upper_limit()
        print(f"✅ get_bounded_position_upper_limit() works: {upper_limit}")
    except Exception as e:
        print(f"❌ get_bounded_position_upper_limit() failed: {e}")

    # 8. Test get_bounded_velocity_lower_limit
    try:
        velocity_lower = iiwa.get_bounded_velocity_lower_limit()
        print(f"✅ get_bounded_velocity_lower_limit() works: {velocity_lower}")
    except Exception as e:
        print(f"❌ get_bounded_velocity_lower_limit() failed: {e}")

    # 9. Test get_bounded_velocity_upper_limit
    try:
        velocity_upper = iiwa.get_bounded_velocity_upper_limit()
        print(f"✅ get_bounded_velocity_upper_limit() works: {velocity_upper}")
    except Exception as e:
        print(f"❌ get_bounded_velocity_upper_limit() failed: {e}")

    # 10. Test get_robot_position_from_plant_context
    try:
        context = iiwa.get_fresh_plant_context()
        position = iiwa.get_robot_position_from_plant_context(context)
        print(f"✅ get_robot_position_from_plant_context() works: {position}")
    except Exception as e:
        print(f"❌ get_robot_position_from_plant_context() failed: {e}")

    # 11. Test get_robot_velocity_from_plant_context
    try:
        context = iiwa.get_fresh_plant_context()
        velocity = iiwa.get_robot_velocity_from_plant_context(context)
        print(f"✅ get_robot_velocity_from_plant_context() works: {velocity}")
    except Exception as e:
        print(f"❌ get_robot_velocity_from_plant_context() failed: {e}")

    print("\n✅✅✅ All tests completed! ✅✅✅")

# Initialize the environment and IIWA object
try:
    playground = Playground(meshcat=meshcat)
    playground = Playground(meshcat=meshcat)
    iiwa = IIWA(playground.construct_welded_sim(playground.default_continuous_state()))
    test_iiwa_functions(iiwa)
except Exception as e:
    print(f"❌ Failed to initialize IIWA: {e}")

/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
Plant init:  [ 0.5   0.6   0.   -1.75  0.    1.    0.    0.    0.    1.    0.    0.
  0.    0.    0.    0.  ]
Directives:  [ModelDirective(add_model=AddModel(file='package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf', name='iiwa', default_joint_positions={'iiwa_joint_1': array([0.]), 'iiwa_joint_2': array([0.6]), 'iiwa_joint_3': array([0.]), 'iiwa_joint_4': array([-1.75]), 'iiwa_joint_5': array([0.]), 'iiwa_joint_6': array([1.]), 'iiwa_joint_7': array([0.])}, default_free_body_pose={}), add_model_instance=None, add_frame=None, add_weld=None, add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=None, add_model_instance=None, add_frame=None, add_weld=AddWeld(parent='world', child='iiwa::iiwa_link_0', X_PC=None), add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=AddModel(file='package://manipulation/hydro/schunk_wsg_50_with_tip.sdf', name='wsg', defau

In [4]:
from planner import MoveToGrasp
playground = Playground(meshcat=meshcat)
mb = playground.env.movable_bodies[0]
pregrasp_move = MoveToGrasp(playground, mb)
pregrasp_move.state_init()
done = False
while not done:
    _, done = pregrasp_move.run()


/home/ronaldocd/Desktop/IMARO/stage/Project/package.xml
parse
Plant init:  [ 0.5   0.6   0.   -1.75  0.    1.    0.    0.    0.    1.    0.    0.
  0.    0.    0.    0.  ]
Directives:  [ModelDirective(add_model=AddModel(file='package://drake_models/iiwa_description/sdf/iiwa7_with_box_collision.sdf', name='iiwa', default_joint_positions={'iiwa_joint_1': array([0.]), 'iiwa_joint_2': array([0.6]), 'iiwa_joint_3': array([0.]), 'iiwa_joint_4': array([-1.75]), 'iiwa_joint_5': array([0.]), 'iiwa_joint_6': array([1.]), 'iiwa_joint_7': array([0.])}, default_free_body_pose={}), add_model_instance=None, add_frame=None, add_weld=None, add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=None, add_model_instance=None, add_frame=None, add_weld=AddWeld(parent='world', child='iiwa::iiwa_link_0', X_PC=None), add_collision_filter_group=None, add_directives=None), ModelDirective(add_model=AddModel(file='package://manipulation/hydro/schunk_wsg_50_with_tip.sdf', name='wsg', defau

TypeError: SetPositionsAndVelocities(): incompatible function arguments. The following argument types are supported:
    1. (self: pydrake.multibody.plant.MultibodyPlant, context: pydrake.systems.framework.Context, q_v: numpy.ndarray[numpy.float64[m, 1]]) -> None
    2. (self: pydrake.multibody.plant.MultibodyPlant, context: pydrake.systems.framework.Context, model_instance: pydrake.multibody.tree.ModelInstanceIndex, q_v: numpy.ndarray[numpy.float64[m, 1]]) -> None

Invoked with: <pydrake.multibody.plant.MultibodyPlant object at 0x7057fc0af290>, <pydrake.systems.framework.LeafContext object at 0x70580834fcf0>, None

In [ ]:
for i, q in enumerate(pregrasp_move.path):
    print(q)

(0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0)
(1.418740306135795e-06, 0.5642267480939203, 6.283184031303833, -1.7494987606061063, -6.175061567798239e-07, 0.9749236216595253, 8.510685981822132e-08, -0.0004311740791859032, 0.00043089488633158785)
(2.83748061227159e-06, 0.5284534961878404, 6.28318275542808, -1.7489975212122124, -1.2350123135596477e-06, 0.9498472433190506, 1.7021371963644264e-07, -0.0008623481583718064, 0.0008617897726631757)
(4.2562209184073854e-06, 0.4926802442817607, 6.283181479552327, -1.7484962818183187, -1.8525184703394717e-06, 0.924770864978576, 2.5532057945466396e-07, -0.0012935222375577097, 0.0012926846589947636)
(5.67496122454318e-06, 0.45690699237568094, 6.283180203676573, -1.747995042424425, -2.4700246271192954e-06, 0.8996944866381013, 3.404274392728853e-07, -0.0017246963167436128, 0.0017235795453263514)
(7.093701530678975e-06, 0.42113374046960117, 6.2831789278008205, -1.7474938030305311, -3.087530783899119e-06, 0.8746181082976265, 4.255342990911066e-07, -0.0